## 라마 3 토크나이저

공개된 LLM을 불러올 때는 transformers 패키지를 이용함.

In [1]:
from transformers import AutoTokenizer

/opt/anaconda3/envs/llm_study/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


AutoTokenizer.from_pretrained("모델이름")을 이용하여 토크나이저를 불러옴.

https://huggingface.co/allganize/Llama-3-Alpha-Ko-8B-Instruct

In [2]:
model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer 는 인코딩과 디코딩을 해준다.

In [4]:
tokenizer.encode('안녕하세요, 반갑습니다.')

[128000, 101193, 124409, 11, 64857, 115072, 39331, 13]

In [5]:
tokenizer.decode([128000, 101193, 124409, 11, 64857, 115072, 39331, 13])

'<|begin_of_text|>안녕하세요, 반갑습니다.'

# Template

모델이 학습되었을 당시의 모델의 입력으로 사용된 형식

In [15]:
messages = [
    {"role": "system", "content": "당신은 친절한 AI 도우미 입니다."},
    {"role": "user", "content": "은행의 기준 금리에 대해서 설명해줘"}
]

'tokenizer.apply_chat_template' 를 통해서 학습 시의 형식을 몰라도 바꾸어 준다.

In [16]:
template_messages = tokenizer.apply_chat_template(messages, tokenize=False)
print(template_messages)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 친절한 AI 도우미 입니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

은행의 기준 금리에 대해서 설명해줘<|eot_id|>


In [17]:
template_messages = tokenizer.apply_chat_template(messages, tokenize=True)
print(template_messages)

[128000, 128006, 9125, 128007, 271, 65895, 83628, 34804, 108280, 104834, 24486, 15592, 101703, 41381, 57139, 119519, 13, 128009, 128006, 882, 128007, 271, 34804, 101066, 21028, 111902, 104193, 111833, 122115, 114942, 34983, 59269, 246, 128009]


In [18]:
encodeds = tokenizer.apply_chat_template(messages)
print('템플릿 적용 및 정수 인코딩 후:\n', encodeds)
print('--' * 100)
print('템플릿 적용 및 정수 인코딩 결과를 복원:\n',tokenizer.decode((encodeds)))

템플릿 적용 및 정수 인코딩 후:
 [128000, 128006, 9125, 128007, 271, 65895, 83628, 34804, 108280, 104834, 24486, 15592, 101703, 41381, 57139, 119519, 13, 128009, 128006, 882, 128007, 271, 34804, 101066, 21028, 111902, 104193, 111833, 122115, 114942, 34983, 59269, 246, 128009]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
템플릿 적용 및 정수 인코딩 결과를 복원:
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 친절한 AI 도우미 입니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

은행의 기준 금리에 대해서 설명해줘<|eot_id|>


# 어시스턴트 시작점

학습 입력에 따라서 답변 생성 직전까지의 토큰까지 넣어줘야 답변을 생성하게 된다.

In [19]:
encodeds = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
print(encodeds)
print('템플릿 적용 및 정수 인코딩 결과를 복원:\n',tokenizer.decode((encodeds)))

[128000, 128006, 9125, 128007, 271, 65895, 83628, 34804, 108280, 104834, 24486, 15592, 101703, 41381, 57139, 119519, 13, 128009, 128006, 882, 128007, 271, 34804, 101066, 21028, 111902, 104193, 111833, 122115, 114942, 34983, 59269, 246, 128009, 128006, 78191, 128007, 271]
템플릿 적용 및 정수 인코딩 결과를 복원:
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 친절한 AI 도우미 입니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

은행의 기준 금리에 대해서 설명해줘<|eot_id|><|start_header_id|>assistant<|end_header_id|>




# Qwen2 템플릿 확인

In [20]:
model_id = "Qwen/Qwen2-7B-Instruct"
qwen_tokenizer = AutoTokenizer.from_pretrained(model_id)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
device = "cuda:0"
messages = [
    {"role": "system", "content": "당신은 패스트캠퍼스 챗봇 패캠이입니다. 묻는 말에 친절하고 정확하게 답변하세요."},
    {"role": "user", "content": "당신이 누군지 설명해주십시오"}
]

encodeds = qwen_tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)
model_inputs = encodeds.to(device)

In [ ]:
print(qwen_tokenizer.decode(model_inputs[0]))